# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import re
import numpy as np
import pickle
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score, precision_score
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///ETLDB.db')
df = pd.read_sql("SELECT * FROM ETLTable", engine)
X = df.message.values
Y = df.iloc[:,4:].values

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)

### ALTERNATE IMPLEMENTATION ###
#Y_test_split = np.split(Y_test,indices_or_sections=36,axis=1)
#Y_pred_split = np.split(Y_pred,indices_or_sections=36,axis=1)

#for i in range(0, len(Y_test_split)):
#    print(classification_report((Y_test_split[i]), (Y_pred_split[i])))

for col in range(Y_test.shape[1]):
    print("Column Name is: ", df.iloc[:,4:].columns[col])
    print("Classification Report is: \n")
    print(classification_report((Y_test[:, col]), (Y_pred[:, col])))
    print("Confusion Matrix is: \n")
    print(confusion_matrix(Y_test[:, col], Y_pred[:, col]))
    print("Accuracy is: \n")
    print((Y_pred[:, col] == Y_test[:, col]).mean())




Column Name is:  related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.63      0.36      0.46      1536
          1       0.82      0.94      0.87      4973
          2       0.60      0.08      0.15        36

avg / total       0.78      0.80      0.77      6545

Confusion Matrix is: 

[[ 552  983    1]
 [ 320 4652    1]
 [   2   31    3]]
Accuracy is: 

0.795569136746
Column Name is:  request
Classification Report is: 

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5430
          1       0.82      0.37      0.51      1115

avg / total       0.87      0.88      0.86      6545

Confusion Matrix is: 

[[5339   91]
 [ 702  413]]
Accuracy is: 

0.878838808251
Column Name is:  offer
Classification Report is: 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        23

avg / total

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[6326    3]
 [ 211    5]]
Accuracy is: 

0.96730328495
Column Name is:  death
Classification Report is: 

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6261
          1       0.74      0.17      0.28       284

avg / total       0.95      0.96      0.95      6545

Confusion Matrix is: 

[[6244   17]
 [ 235   49]]
Accuracy is: 

0.961497326203
Column Name is:  other_aid
Classification Report is: 

             precision    recall  f1-score   support

          0       0.87      1.00      0.93      5669
          1       0.64      0.03      0.06       876

avg / total       0.84      0.87      0.81      6545

Confusion Matrix is: 

[[5653   16]
 [ 847   29]]
Accuracy is: 

0.868143621085
Column Name is:  infrastructure_related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6091
          1       0.30      0.01      0.01       454

avg / total       0

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f109e35d598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [8]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0)
}

cv = GridSearchCV(pipeline, param_grid=parameters)  
cv.fit(X_train, Y_train)
Y_pred_grid_search = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
for col in range(Y_test.shape[1]):
    print("Column Name is: ", df.iloc[:,4:].columns[col])
    print("Classification Report is: \n")
    print(classification_report((Y_test[:, col]), (Y_pred_grid_search[:, col])))
    print("Confusion Matrix is: \n")
    print(confusion_matrix(Y_test[:, col], Y_pred_grid_search[:, col]))
    print("Accuracy is: \n")
    print((Y_pred_grid_search[:, col] == Y_test[:, col]).mean())


Column Name is:  related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.61      0.33      0.43      1536
          1       0.81      0.93      0.87      4973
          2       0.67      0.06      0.10        36

avg / total       0.76      0.79      0.76      6545

Confusion Matrix is: 

[[ 507 1028    1]
 [ 328 4645    0]
 [   2   32    2]]
Accuracy is: 

0.787471352177
Column Name is:  request
Classification Report is: 

             precision    recall  f1-score   support

          0       0.89      0.99      0.93      5430
          1       0.85      0.38      0.53      1115

avg / total       0.88      0.88      0.86      6545

Confusion Matrix is: 

[[5352   78]
 [ 688  427]]
Accuracy is: 

0.882964094729
Column Name is:  offer
Classification Report is: 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        23

avg / total

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Accuracy is: 

0.968067226891
Column Name is:  death
Classification Report is: 

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6261
          1       0.78      0.10      0.17       284

avg / total       0.95      0.96      0.94      6545

Confusion Matrix is: 

[[6253    8]
 [ 256   28]]
Accuracy is: 

0.959663865546
Column Name is:  other_aid
Classification Report is: 

             precision    recall  f1-score   support

          0       0.87      1.00      0.93      5669
          1       0.56      0.03      0.05       876

avg / total       0.83      0.87      0.81      6545

Confusion Matrix is: 

[[5650   19]
 [ 852   24]]
Accuracy is: 

0.86692131398
Column Name is:  infrastructure_related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6091
          1       0.33      0.00      0.01       454

avg / total       0.89      0.93      0.90   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
# Use KNeighborsClassifier 
kn_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
kn_pipeline.fit(X_train, Y_train)
Y_pred_kn = kn_pipeline.predict(X_test)

In [12]:
for col in range(Y_test.shape[1]):
    print("Column Name is: ", df.iloc[:,4:].columns[col])
    print("Classification Report is: \n")
    print(classification_report((Y_test[:, col]), (Y_pred_kn[:, col])))
    print("Confusion Matrix is: \n")
    print(confusion_matrix(Y_test[:, col], Y_pred_kn[:, col]))
    print("Accuracy is: \n")
    print((Y_pred_kn[:, col] == Y_test[:, col]).mean())


Column Name is:  related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.52      0.41      0.46      1540
          1       0.82      0.88      0.85      4960
          2       0.71      0.11      0.19        45

avg / total       0.75      0.77      0.75      6545

Confusion Matrix is: 

[[ 635  905    0]
 [ 589 4369    2]
 [   5   35    5]]
Accuracy is: 

0.765317035905
Column Name is:  request
Classification Report is: 

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5427
          1       0.77      0.33      0.46      1118

avg / total       0.86      0.87      0.85      6545

Confusion Matrix is: 

[[5317  110]
 [ 751  367]]
Accuracy is: 

0.868449197861
Column Name is:  offer
Classification Report is: 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6517
          1       0.00      0.00      0.00        28

avg / total

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[6175   97]
 [ 229   44]]
Accuracy is: 

0.950190985485
Column Name is:  other_aid
Classification Report is: 

             precision    recall  f1-score   support

          0       0.87      0.99      0.93      5681
          1       0.42      0.04      0.07       864

avg / total       0.81      0.87      0.81      6545

Confusion Matrix is: 

[[5637   44]
 [ 832   32]]
Accuracy is: 

0.86615737204
Column Name is:  infrastructure_related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.94      0.99      0.97      6177
          1       0.13      0.03      0.05       368

avg / total       0.90      0.93      0.91      6545

Confusion Matrix is: 

[[6105   72]
 [ 357   11]]
Accuracy is: 

0.934453781513
Column Name is:  transport
Classification Report is: 

             precision    recall  f1-score   support

          0       0.96      0.99      0.98      6256
          1       0.35      0.08      0.13       289

avg / total    

In [13]:
# Use custom transformer to provide feature for emergency/urgent messages
class EmergencyMessageExtractor(BaseEstimator, TransformerMixin):

    def emergency_message(self, text):
        if "emergency" in text.lower() or "urgent" in text.lower():
            return True
        else:
            return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.emergency_message)
        return pd.DataFrame(X_tagged)    

In [14]:
# Pipeline with custom emergency message extractor
pipeline_with_emergency_extractor = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('emergency_message', EmergencyMessageExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline_with_emergency_extractor.fit(X_train, Y_train)
Y_pred_with_emergency_extractor = pipeline_with_emergency_extractor.predict(X_test)

In [15]:
for col in range(Y_test.shape[1]):
    print("Column Name is: ", df.iloc[:,4:].columns[col])
    print("Classification Report is: \n")
    print(classification_report((Y_test[:, col]), (Y_pred_with_emergency_extractor[:, col])))
    print("Confusion Matrix is: \n")
    print(confusion_matrix(Y_test[:, col], Y_pred_with_emergency_extractor[:, col]))
    print("Accuracy is: \n")
    print((Y_pred_with_emergency_extractor[:, col] == Y_test[:, col]).mean())

Column Name is:  related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.63      0.34      0.44      1545
          1       0.82      0.94      0.87      4956
          2       0.50      0.23      0.31        44

avg / total       0.77      0.79      0.77      6545

Confusion Matrix is: 

[[ 529 1010    6]
 [ 315 4637    4]
 [   1   33   10]]
Accuracy is: 

0.790832696715
Column Name is:  request
Classification Report is: 

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5422
          1       0.85      0.41      0.55      1123

avg / total       0.88      0.89      0.87      6545

Confusion Matrix is: 

[[5339   83]
 [ 668  455]]
Accuracy is: 

0.88525592055
Column Name is:  offer
Classification Report is: 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6521
          1       0.00      0.00      0.00        24

avg / total 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Column Name is:  other_aid
Classification Report is: 

             precision    recall  f1-score   support

          0       0.87      1.00      0.93      5689
          1       0.53      0.04      0.07       856

avg / total       0.83      0.87      0.82      6545

Confusion Matrix is: 

[[5662   27]
 [ 826   30]]
Accuracy is: 

0.869671504966
Column Name is:  infrastructure_related
Classification Report is: 

             precision    recall  f1-score   support

          0       0.93      1.00      0.97      6109
          1       0.00      0.00      0.00       436

avg / total       0.87      0.93      0.90      6545

Confusion Matrix is: 

[[6106    3]
 [ 436    0]]
Accuracy is: 

0.932925897632
Column Name is:  transport
Classification Report is: 

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6254
          1       0.94      0.06      0.11       291

avg / total       0.96      0.96      0.94      6545

Confusion Matrix 

### 9. Export your model as a pickle file

In [17]:
pickle.dump(cv, open('./grid_search_clf.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.